# Convert csv to parquet

In [2]:
from util import *

In [2]:
# Convert all csv files to parquet or vice versa
files = os.listdir('../data')
for file in files:
    if not 'trans' in file: continue
    
    if file.endswith('.csv'):
        convert_csv_to_parquet('../data/' + file)
    elif file.endswith('.parquet'):
        convert_parquet_to_csv('../data/' + file)

KeyboardInterrupt: 